In [1]:
import sys
import tqdm
import pandas as pd
import sys
import tqdm
import pandas as pd
import os
from string import Template
from SPARQLWrapper import SPARQLWrapper, JSON, CSV


In [7]:
df=pd.read_csv('wiki_Qid.csv')
names_from_revisions=pd.read_csv('NER_results.csv')

# Creating a SPARQL Query to get information of the entities we obtained from Flair NER

In [3]:
def query(person_wd, folder=''):
    filename = os.path.join(folder, str(person_wd) + ".csv")
    
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    
    endpoint_url = "https://query.wikidata.org/sparql"
    query_template = Template("""select Distinct ?genderLabel ?nameLabel ?person ?nametype ?personLabel 
WHERE {  
  BIND($person_wd AS ?person)
  { # get "also known as" in most prominent languages
  ?person skos:altLabel ?altLabel  FILTER(lang(?altLabel) IN ("en","de","es","fr","it","pl","pt")) .
  # Remove language tag from ?altLabel to allow distinct 
  BIND (STR(?altLabel)  AS ?nameLabel) 
  BIND ("alternate" as ?nametype)
}
UNION
{ # get givenname
    ?person wdt:P735 ?name
    BIND ("given name" as ?nametype)
}
UNION
{ # get nickname
  ?person wdt:P1449 ?name
  BIND ("nickname" as ?nametype)
}
UNION
{ # get family name
  ?person wdt:P734 ?name
  BIND ("family name" as ?nametype)
}
UNION
{ # get label of person
  ?person rdfs:label ?name FILTER(lang(?name) IN ("en","de","es","fr","it","pl","pt"))
  BIND ("name" as ?nametype)
}  
optional{ ?person  wdt:P21 ?gender.}  
BIND(coalesce(?gender, "no") as ?gender). 
# set language for labels
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
""")
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    query = query_template.substitute(person_wd=person_wd)
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df=pd.json_normalize(results['results']['bindings']) #json to csv
    results_df=results_df[['genderLabel.value','nameLabel.value','person.value','nametype.value', 'personLabel.value']]
    results_df.columns= ['gender', 'nameLabel', 'person','nametype','personLabel']
    results_df.to_csv(filename, index=False)


In [4]:
wikidata_names = pd.concat([query(person_wd=person_wd) for person_wd in list(df['wiki_Qid'])])

In [19]:
wikidata_names=wikidata_names[wikidata_names['gender'] != 'no'] 

In [63]:

result = []
for idx, row in tqdm.tqdm(names_from_revisions.iterrows(), total=names_from_revisions.shape[0]):    
    linked = False
    for idx2, row2 in wikidata_names.iterrows():
        if row['index'] == row2['nameLabel']:
            result.append({'Name':row['index'], 
                           'Count':row['0'],
                           'Revision':row['revision'], 
                           'link':row2['person'],
                           'gender':row2['gender'],
                          'person' : row2['personLabel']})
            linked = True
            continue
    if not linked:
        result.append({'Name':row['index'],
                       'Count':row['0'],
                       'Revision':row['revision'],
                       'link':'not yet linked',
                       'gender':'unknown',
                      'person':'unknown'})
Wiki_gender = pd.DataFrame(result)

100%|████████████████████████████████████████████████████████████████████████████████| 411/411 [00:06<00:00, 64.28it/s]


,Name,Count,Revision,link,gender,person
0,Paul Tibbets,1.0,2,http://www.wikidata.org/entity/Q294555,male,Paul Tibbets
1,Paul W Tibbets Jr,1.0,2,not yet linked,unknown,unknown
2,Charles Sweeney,1.0,2,http://www.wikidata.org/entity/Q241128,male,Charles Sweeney
3,Paul W Tibbets,3.0,2,not yet linked,unknown,unknown
4,Thomas J Classen,3.0,2,not yet linked,unknown,unknown
...,...,...,...,...,...,...
438,LeMay Curtis,1.0,9,not yet linked,unknown,unknown
439,Mann Robert,1.0,9,not yet linked,unknown,unknown
440,Marx Joseph,1.0,9,not yet linked,unknown,unknown
441,Thomas Gordon,2.0,9,http://www.wikidata.org/entity/Q3525168,male,Thomas Gordon
